Apparently Log_softmax is causing NaN(Not a Number) error after multiple epochs. 
This is because we are doing the following:

In [1]:
out_matrix = self.matrix - np.log(np.sum(np.exp(self.matrix), axis=axis, keepdims=True))

NameError: name 'self' is not defined

We see a crash at "np.exp(self.matrix)" because as the predictions start to become accurate, the logits are high and thus their exponentials become infinity. 

thus we can apply a simple fix by subtracting down all the values in the matrix by the max term along their axis.

i.e.

In [ ]:
def log_softmax(self, axis=-1):
        #Max value in the batch
        c = np.max(self.matrix, axis=axis, keepdims=True)
        
        #Subtract max BEFORE exponentiating
        log_sum_exp = c + np.log(np.sum(np.exp(self.matrix - c), axis=axis, keepdims=True))
        
        out_matrix = self.matrix - log_sum_exp
        
        return tensor(out_matrix) 

To fix high memory usage, we have to redesign some parts. We have been simply using out.grad in our _backward() lamdas which cause circular referencing. Out own's the lamda _backward() which owns out. Thus, neither of them get cleared out. Thus, we need to decouple them by passing out.grad as an argument to _backward(). Yes! This does mean we have to change all our operations. But, this will be worth it. We will neither need Garbage Collector calls, nor weak referencing.